In [1]:
import pandas as pd
import time

In [4]:
filepath="../../dataset/anomaly_detection_1/train.csv"
data=pd.read_csv(filepath)

In [7]:
KPI_ID = pd.unique(data['KPI ID'])

In [8]:
KPI_ID

array(['02e99bd4f6cfb33f', '9bd90500bfd11edb', 'e0770391decc44ce',
       'da403e4e3f87c9e0', 'a5bf5d65261d859a', '18fbb1d5a5dc099d',
       '8c892e5525f3e491', 'e5e3cd1a03fee6bd', '40e25005ff8992bd',
       'cff6d3c01e6a6bfa', '71595dd7171f4540', '09513ae3e75778a3',
       '9ee5879409dccef9', 'c58bfcbacb2822d1', '1c35dbf57f55f5e4',
       '88cf3a776ba00e7c', '046ec29ddf80d62e', '7c189dd36f048a6c',
       'a40b1df87e3f1c87', '8bef9af9a922e0b3', 'affb01ca2b4f0b45',
       '07927a9a18fa19ae', '54e8a140f6237526', 'b3b2e6d1a791d63a',
       '8a20c229e9860d0c', '769894baefea4e9e', '76f4550c43334374'], dtype=object)

In [11]:
KPI_ID00 = data[data['KPI ID']==KPI_ID[0]]
KPI_ID01 = data[data['KPI ID']==KPI_ID[1]]
KPI_ID02 = data[data['KPI ID']==KPI_ID[2]]
KPI_ID03 = data[data['KPI ID']==KPI_ID[3]]
KPI_ID04 = data[data['KPI ID']==KPI_ID[4]]
KPI_ID05 = data[data['KPI ID']==KPI_ID[5]]
KPI_ID06 = data[data['KPI ID']==KPI_ID[6]]
KPI_ID07 = data[data['KPI ID']==KPI_ID[7]]
KPI_ID08 = data[data['KPI ID']==KPI_ID[8]]
KPI_ID09 = data[data['KPI ID']==KPI_ID[9]]
KPI_ID10 = data[data['KPI ID']==KPI_ID[10]]
KPI_ID11 = data[data['KPI ID']==KPI_ID[11]]
KPI_ID12 = data[data['KPI ID']==KPI_ID[12]]
KPI_ID13 = data[data['KPI ID']==KPI_ID[13]]
KPI_ID14 = data[data['KPI ID']==KPI_ID[14]]
KPI_ID15 = data[data['KPI ID']==KPI_ID[15]]
KPI_ID16 = data[data['KPI ID']==KPI_ID[16]]
KPI_ID17 = data[data['KPI ID']==KPI_ID[17]]
KPI_ID18 = data[data['KPI ID']==KPI_ID[18]]
KPI_ID19 = data[data['KPI ID']==KPI_ID[19]]
KPI_ID20 = data[data['KPI ID']==KPI_ID[20]]
KPI_ID21 = data[data['KPI ID']==KPI_ID[21]]
KPI_ID22 = data[data['KPI ID']==KPI_ID[22]]
KPI_ID23 = data[data['KPI ID']==KPI_ID[23]]
KPI_ID24 = data[data['KPI ID']==KPI_ID[24]]
KPI_ID25 = data[data['KPI ID']==KPI_ID[25]]
KPI_ID26 = data[data['KPI ID']==KPI_ID[26]]

In [15]:
KPI_ID00.head()

,KPI ID,timestamp,value,label
0,02e99bd4f6cfb33f,1493568000,1.901639,0
1,02e99bd4f6cfb33f,1493568060,1.786885,0
2,02e99bd4f6cfb33f,1493568120,2.000000,0
3,02e99bd4f6cfb33f,1493568180,1.885246,0
4,02e99bd4f6cfb33f,1493568240,1.819672,0


In [103]:
length = len(KPI_ID01)
ano_num= sum(KPI_ID00.label)
[length,ano_num]

[131795, 2131]

In [82]:
samples = make_samples(KPI_ID00['value'],KPI_ID00['label'],20)
samples.head()

,before0,before1,before10,before11,before12,before13,before14,before15,before16,before17,...,before2,before20,before3,before4,before5,before6,before7,before8,before9,label
0,2.180328,2.032787,1.950820,1.950820,1.967213,1.934426,1.885246,1.885246,1.819672,1.885246,...,1.967213,1.901639,2.049180,1.918033,1.836066,1.803279,1.934426,1.901639,1.950820,0
1,1.934426,2.180328,1.950820,1.950820,1.950820,1.967213,1.934426,1.885246,1.885246,1.819672,...,2.032787,1.786885,1.967213,2.049180,1.918033,1.836066,1.803279,1.934426,1.901639,0
2,1.950820,1.934426,1.901639,1.950820,1.950820,1.950820,1.967213,1.934426,1.885246,1.885246,...,2.180328,2.000000,2.032787,1.967213,2.049180,1.918033,1.836066,1.803279,1.934426,0
3,1.934426,1.950820,1.934426,1.901639,1.950820,1.950820,1.950820,1.967213,1.934426,1.885246,...,1.934426,1.885246,2.180328,2.032787,1.967213,2.049180,1.918033,1.836066,1.803279,0
4,1.983607,1.934426,1.803279,1.934426,1.901639,1.950820,1.950820,1.950820,1.967213,1.934426,...,1.950820,1.819672,1.934426,2.180328,2.032787,1.967213,2.049180,1.918033,1.836066,0


In [110]:
samples_train = samples[samples.label==0]
[len(samples),len(samples_train)]

[131775, 129644]

In [111]:
#Isolation Forest
from sklearn.ensemble import IsolationForest

clf = IsolationForest(n_estimators=500,max_samples = 0.005)
clf.fit(samples_train.iloc[:,0:20])
y_pred_sampes=clf.predict(samples.iloc[:,0:20])
y_pred_sampes

array([1, 1, 1, ..., 1, 1, 1])

In [115]:
sum(y_pred_sampes+samples.label==0)

671

In [79]:
#algorith1  isolatation forecast!!  长度为x

def make_samples(series,anomaly_series,period):
    if period>len(series):
        return "error"
    result={}
    for start in range(period,len(series)):
        result = append_data(result,"label",anomaly_series[start])
        strs = getStrs("before",period+1)
        for i in range(period+1):
            result = append_data(result,strs[i],series[start-i])
    return pd.DataFrame(result)

def append_data(dic,key,value):
    if(dic.has_key(key)):
        dic[key].append(value)
    else:
        dic[key] = [value]
    return dic

def getStrs(pre,num):
    result = []
    for i in range(num):
        result.append(pre+str(i))
    return result

In [80]:
result=make_samples([1,2,3,4,5,6,7,8],[1,1,1,0,1,0,1,1],3)

In [81]:
result

,before0,before1,before2,before3,label
0,4,3,2,1,0
1,5,4,3,2,1
2,6,5,4,3,0
3,7,6,5,4,1
4,8,7,6,5,1


str